# TESTS

Build pandas dataframe with all track from all CDs

In [252]:
# imports
import pandas as pd
import os
import glob

#initial values -- REQUIRED TO BE SET
audio_file_type = 'mp3' #initialization

# func def
def cd_number(audio_dir):
    """
    Extracts from a directory path the CDXX number and returns it
    """
    import re #regular expression to search for CDXX format where XX any combination of natural numbers eg 00, 45, etc
    audio_dir = audio_dir.split('/')[-1]
    x = re.search(r"CD[0-9][0-9]", audio_dir)
    return x.group()

def get_cd_img_file(cd_path):
    """
    Input: CD path
    Returns: path of the jpg file in the CD directory that contains the list of songs for that particular CD
    """
    images_list = glob.glob(f'{cd_path}/*.jpg')
    return fix_path(images_list) #just a single imagefile
    
def track_number(audio_path):
    """
    Input: audio file path
    Returns: Number of track in CD
    """
    #return audio_path.split('/')[-1][:2]
    import re
    x = re.search(r"[0-9][0-9]", audio_path.split('/')[-1])
    return x.group()

def fix_path(my_list):
    """
    Input: list of paths or single string path with Windows format (\\ instead of /)
    Output: fixed list of paths or single string containing only /
    """
    def replace_slash(path):
        '''replcae double backslashes in Windows paths to forward ones (compatible with UNIX)'''
        new_path = path.replace('\\', '/')
        return new_path
    
    if isinstance(my_list, list): #if it's a list return the fixed list of items
        my_new_list = []
        for item in my_list:
            fixed_item = replace_slash(item)
            my_new_list.append(fixed_item)

        return my_new_list
    
    elif isinstance(my_list, str): #if it's a string just return the fixed one
        return replace_slash(my_list)

    
def add_1000_cover_jpg_file_to_all_cds(df):
    """
    adds a 1000 Cover.jpg file to all existing CD paths if it's not already there
    This is useful to parse then the jpg files along with the mp3s to create video files combining the mp3s and jpgs
    """
    import shutil, os

    image_to_add = f"{df.loc[0, 'CD path']}/1000  Cover.jpg"

    for i in range(len(df)):
        cd_path = df.loc[i, 'CD path']

        #add the 1000 cover file if not already there
        if not os.path.isfile(f"{df.loc[i, 'CD path']}/1000  Cover.jpg"):
            shutil.copy(image_to_add, cd_path)    

            
current_working_dir = os.getcwd() #eg 'C:\\Users\\User\\Downloads\\Mp3toMp4'
current_working_dir = fix_path(current_working_dir) # CONVERT WINDOWS PATH TO UNIVERSAL PATH

audio_dir_suffix = 'Audio_tracks' #directory within our current working directory that audio files exist
audio_dir= fix_path(f'{current_working_dir}/{audio_dir_suffix}')

inner_dir = glob.glob(f'{audio_dir}/*') #get any subdir
inner_dir = fix_path(inner_dir)

for item in inner_dir: #for each inner dir search for final (last) dirs
    all_audio_dirs = glob.glob(f'{item}/*')
    all_audio_dirs = fix_path(all_audio_dirs)

cd = []
track = []
cd_path = []
track_path = []
cd_songs_list_img = []

for current_cd_path in all_audio_dirs: #for each CD path in the list of all CD paths
    cd_num = cd_number(current_cd_path)
    cd_audio_tracks = glob.glob(f'{current_cd_path}/*.{audio_file_type}') #gets all filetypes specified eg *.mp3
    cd_audio_tracks = fix_path(cd_audio_tracks)
    cd_list_img = get_cd_img_file(current_cd_path)
    
    for audio_track in cd_audio_tracks: #for each track within each CD
        cd.append(cd_num)
        track.append(track_number(audio_track))
        cd_path.append(current_cd_path)
        track_path.append(audio_track)
        cd_songs_list_img.append(cd_list_img)

df = pd.DataFrame({'CD' : cd,
                   'track' : track,
                   'CD path': cd_path,
                   'track path': track_path,
                   'CD songs list jpg': cd_songs_list_img
                  })


In [253]:
df

,CD,track,CD path,track path,CD songs list jpg
0,CD01,01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]"
1,CD01,02,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]"
2,CD01,03,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/03 - ---------S S-T--S--.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]"
3,CD01,04,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/04 - ---- S- --------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]"
4,CD01,05,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/05 - ------S -- --- ------ -----.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]"
...,...,...,...,...,...
957,CD52,09,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/09 - Un fiume amaro (- -a-µ--) - Iva Zanicchi.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]"
958,CD52,10,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/10 - Varje Morgon (---e p---) - Arja Saijonmaa.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]"
959,CD52,11,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/11 - My love (---p- µ--) - Jocelyn B Smith.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]"
960,CD52,14,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/14 - Ciutat preciosa (-µ--f- p---) - Maria del Mar Bonet.mp3,"[C:/Users/User/Downloads/Mp3

In [254]:
track1 = df.loc[0]
track1

CD                                                                                                                                                                                                                                     CD01
track                                                                                                                                                                                                                                    01
CD path                                                                                                                                       C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01
track path                                                                                           C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3
CD songs list jpg    [C:/Users/User/Downloads/Mp3toMp4/A

In [255]:
from mutagen.mp3 import MP3

track_title = []
track_singer = []
mp3_items = []

for i in range(len(df)):
    track1 = df.loc[i]
    track1_mp3 = MP3(track1['track path'])
    track_title.append(track1_mp3['TIT2'])
    track_singer.append(track1_mp3['TPE1'])
    mp3_items.append(track1_mp3)
    
df['title'] = track_title
df['singer'] = track_singer
df['mp3'] = mp3_items

In [256]:
df

,CD,track,CD path,track path,CD songs list jpg,title,singer,mp3
0,CD01,01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
1,CD01,02,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
2,CD01,03,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/03 - ---------S S-T--S--.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
3,CD01,04,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/04 - ---- S- --------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΟΤΑΝ ΣΕ ΒΡΗΚΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
4,CD01,05,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/05 - ------S -- --- ------ -----.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΕΠΑΙΞΕΣ ΜΕ ΤΗΝ ΚΑΡΔΙΑ ΜΟΥ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
...,...,...,...,...,...,...,...,...
957,CD52,09,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/09 - Un fiume amaro (- -a-µ--) - Iva Zanicchi.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]",Un fiume amaro (Ο καημός),Iva Zanicchi,"[TPE2, TIT2, TRCK, TENC, TPE1, TALB, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]"
958,CD52,10,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/10 - Varje Morgon (---e p---) - Arja Saijonmaa.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis The

# WILL TRY TO CREATE A VIDEO FROM MP3

In [263]:
def parse_track(track_id, df, videos_dir):
    song_cd = df.loc[track_id, 'CD']
    song_track_num = df.loc[track_id, 'track']
    song_title = df.loc[track_id, 'title']
    song_singer = df.loc[track_id, 'singer']

    video_title = f'{song_title} - {song_singer}'
    video_description = f'1000 τραγούδια του Μίκη Θεοδωράκη\n\
                        {song_cd}\n\
                        Track {song_track_num}\n\
                        Title: {song_title}\n\
                        Music: Μίκης Θεοδωράκης\n\
                        Singer: {song_singer}\n\n\
                        Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων\
                        της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n\
                        ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.'

    video_path = f'{videos_dir}/{song_cd}TR{song_track_num} {song_title} - {song_singer}.mp4'
    return (video_path, video_title, video_description)

In [264]:
video_path = []
video_title = []
video_description = []
videos_dir = f'{current_working_dir}/Video_tracks' #select the path to save generated videos

for video in range(len(df)):
    path, title, description = parse_track(video, df, videos_dir) 
    video_path.append(path)
    video_title.append(title)
    video_description.append(description)

df['video title'] = video_title
df['video description'] = video_description
df['video path'] = video_path
df['video created'] = None

In [266]:
df.tail()

,CD,track,CD path,track path,CD songs list jpg,title,singer,mp3,video title,video path,video created,video description
957,CD52,09,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/09 - Un fiume amaro (- -a-µ--) - Iva Zanicchi.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]",Un fiume amaro (Ο καημός),Iva Zanicchi,"[TPE2, TIT2, TRCK, TENC, TPE1, TALB, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",Un fiume amaro (Ο καημός) - Iva Zanicchi,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD52TR09 Un fiume amaro (Ο καημός) - Iva Zanicchi.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD52\n Track 09\n Title: Un fiume amaro (Ο καημός)\n Music: Μίκης Θεοδωράκης\n Singer: Iva Zanicchi\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.
958,CD52,10,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/10 - Varje Morgon (---e p---) - Arja Saijonmaa.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]",Varje Morgon (Κάθε πρωί),Arja Saijonmaa,"[TPE2, TIT2, TRCK, TENC, TALB, TPE1, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",Varje Morgon (Κάθε πρωί) - Arja Saijonmaa,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD52TR10 Varje Morgon (Κάθε πρωί) - Arja Saijonmaa.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD52\n Track 10\n Title: Varje Morgon (Κάθε πρωί)\n Music: Μίκης Θεοδωράκης\n Singer: Arja Saijonmaa\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.
959,CD52,11,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/11 - My love (---p- µ--) - Jocelyn B Smith.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD52/CD52.jpg]",My love (Αγάπη μου),Jocelyn B Smith,"[TPE2, TIT2, TENC, TRCK, TALB, TPE1, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",My love (Αγάπη μου) - Jocelyn B Smith,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD52TR11 My love (Αγάπη μου) - Jocelyn B Smith.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD52\n Track 11\n Title: My love (Αγάπη μου)\n Music: Μίκης Θεοδωράκης\n Singer: Jocelyn B Smith\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙ

In [267]:
# Loading all the packages required
from mutagen.mp3 import MP3
from PIL import Image
from pathlib import Path
from moviepy import editor

'''
Creating class MP3ToMP4 which contains methods to convert
an audio to a video using a list of images.
'''

class MP3ToMP4:
    def __init__(self, folder_path, audio_path, video_path_name):
        """
        :param folder_path: contains the path of the root folder.
        :param audio_path: contains the path of the audio (mp3 file).
        :param video_path_name: contains the path where the created
                                video will be saved along with the
                                name of the created video.
        """
        self.folder_path = folder_path
        self.audio_path = audio_path
        self.video_path_name = video_path_name
        # Calling the create_video() method.
        self.create_video()
    def get_length(self):
        """
        This method reads an MP3 file and calculates its length
        in seconds.
        :return: length of the MP3 file
        """
        song = MP3(self.audio_path)
        return int(song.info.length)
    def get_images(self):
        """
        This method reads the filenames of the images present
        in the folder_path of type '.jpg' and stores it in the
        'images' list.
        Then it opens the images, resizes them and appends them
        to another list, 'image_list'
        :return: list of opened images
        """
        path_images = Path(self.folder_path)
        images = list(path_images.glob('*.jpg'))
        image_list = list()
        for image_name in images:
            image = Image.open(image_name).resize((800, 800), Image.ANTIALIAS)
            image_list.append(image)
        return image_list
    def create_video(self):
        """
        This method calls the get_length() and get_images()
        methods internally. It then calculates the duration
        of each frame. After that, it saves all the opened images
        as a gif using the save() method. Finally it calls the
        combine_method()
        :return: None
        """
        length_audio = self.get_length()
        image_list = self.get_images()
        duration = int(length_audio / len(image_list)) * 1000
        image_list[0].save(self.folder_path + "temp.gif",
                           save_all=True,
                           append_images=image_list[1:],
                           duration=duration)
        # Calling the combine_audio() method.
        self.combine_audio()
    def combine_audio(self):
        """
        This method attaches the audio to the gif file created.
        It opens the gif file and mp3 file and then uses
        set_audio() method to attach the audio. Finally, it
        saves the video to the specified video_path_name
        :return: None
        """
        video = editor.VideoFileClip(self.folder_path + "temp.gif")
        audio = editor.AudioFileClip(self.audio_path)
        final_video = video.set_audio(audio)
        final_video.write_videofile(self.video_path_name, fps=60)

if __name__ == '__main__':
    # Taking the input for the paths of the variables mentioned below.
    
    for current_track in range(len(df)): #for all ~1000 songs
        folder_path = df.loc[current_track, 'CD path'] #'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/' its the path of the CD where it will look in there for images
        audio_path = df.loc[current_track, 'track path'] #'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3'

        videos_dir = f'{current_working_dir}/Video_tracks' #select the path to save generated videos
        video_path_name, video_title, video_description = parse_track(current_track, df, videos_dir)

        """
        try:
            MP3ToMP4(folder_path, audio_path, video_path_name) # Invoking the parameterized constructor of the MP3ToMP4 class.
            df.loc[current_track, 'video created'] = True #if the video file is created update the df 'video created' column
        except:
            raise(f'Error while creating the video file for {audio_path}')

        """
        

In [268]:
df.head()

,CD,track,CD path,track path,CD songs list jpg,title,singer,mp3,video title,video path,video created,video description
0,CD01,01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR01 ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 01\n Title: ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: Π.ΠΑΝΔΗΣ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.
1,CD01,02,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR02 ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 02\n Title: ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.
2,CD01,03,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/03 - ---------S S-T--S--.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR03 ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ - Π.ΠΑΝΔΗΣ.mp4,None,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 03\n Title: ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ\n Music: Μίκης Θεοδωράκης\n Singer: Π.ΠΑΝΔΗΣ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθη

## Stixoi.info lookup

In [269]:
#parse around 124500 pages in stixoi.info with lyrics

base_url = "https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=" # eg https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=124500

min_id = 1
max_id = 124574

stixoi_df = pd.DataFrame({'url': None},
                         index=pd.RangeIndex(start=min_id, stop=max_id+1, name='id'))

#construct 124574 urls
for index in range(min_id, max_id+1):
    url = f'{base_url}{index}'
    stixoi_df.loc[index, 'url'] = url


In [270]:
stixoi_df

,url
id,
1,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=1
2,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=2
3,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=3
4,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=4
5,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=5
...,...
124570,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=124570
124571,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=124571
124572,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=124572


In [136]:
from bs4 import BeautifulSoup
import requests
import unicodedata as ud

def title_parse(lyrics_url):
    """
    Looks into stixoi.info lyrics page and extracts the title
    """

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    response = requests.get(lyrics_url, headers = headers) #query the webpage and get the HTML response
    #soup = BeautifulSoup(response.content, 'html.parser') #parse the fetched HTML content using an HTML parser
    
    soup = BeautifulSoup(response.content, 'html.parser')

    lyrics = soup.find('title')
    lyrics = str(lyrics)
    lyrics = lyrics.replace("</title>","").split(": ")[-1].upper()
    d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
    lyrics = ud.normalize('NFD', lyrics).upper().translate(d)
    
    return lyrics

In [101]:
# PROVIDE URL AND GET LYRICS FROM STIXOI.INFO
# SEARCH DIDNT WORK PREVIOUSLY SO SKIPPED UNTIL NOW

def lyrics_parse(lyrics_url):
    """
    Looks into stixoi.info lyrics page
    """
    from bs4 import BeautifulSoup #this library allows to go through HTML doc
    #and exctract the divs needed and get the data we want
    import requests #query a website and return its HTML

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    response = requests.get(lyrics_url, headers = headers) #query the webpage and get the HTML response
    #soup = BeautifulSoup(response.content, 'html.parser') #parse the fetched HTML content using an HTML parser
    
    soup = BeautifulSoup(response.content, 'html.parser')

    lyrics = soup.find_all('meta')
    lyrics = lyrics[-1]
    #print(lyrics)
    
    stixakia = soup.find("lyrics")
    #print(stixakia)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    mydivs = soup.find_all("div", {"class": "lyrics"})
    print(type(mydivs))
    
    a_string = []
    for line in mydivs:            
        lyrics_string = str(line).replace("</br>","").replace("<br/>","").replace("<br>","").replace("</div>", "").split('lyrics">')[1]
        a_string.append(lyrics_string)
        #print(lyrics_string)
    
    return a_string
    
url = "https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=61544"
a_string = lyrics_parse(url)


<class 'bs4.element.ResultSet'>


In [102]:
with open('lyrics_sample.txt', 'w') as f:
    for item in a_string:
        print(item)
        f.write(item.replace("\n",""))

Δεν είμαι βράχος, ούτε βουνό, 
να μην δακρύζω και να μην πονώ, 
αφού μου είπανε για σένα
κατηγορίες ένα σωρό, 
αφού μου είπανε για σένα
κατηγορίες ένα σωρό.

Αναρωτιέμαι γιατί
σε κατακρίνουνε πολλοί
και μου λένε "πώς θα ζήσεις
μ’ αυτό το χαμένο κορμί;".

Αμφιβολία φριχτή
μου σκοτεινιάζει την ψυχή
και φοβάμαι μήπως πάθω
καμιά συμφορά στη ζωή.

Εμένανε η τύχη μου το γράφει να πονώ, 
να πνίγομαι στα δάκρυα, ποτέ να μην χαρώ.

Δεν είμαι βράχος, ούτε βουνό, 
να μην δακρύζω και να μην πονώ, 
αφού μου είπανε για σένα
κατηγορίες ένα σωρό, 
αφού μου είπανε για σένα
κατηγορίες ένα σωρό.

Πώς να μην κλάψω πικρά
με τέτοια λόγια θλιβερά
που ακούω καθε τόσο
για σένα, κρυφή μου χαρά.

Μα σε λατρεύω πολύ
κι αν είναι η μοίρα μου αυτή, 
ας τελειώσει όπως θέλει
για μένα αυτή η ζωή.

Εμένανε η τύχη μου το γράφει να πονώ, 
να πνίγομαι στα δάκρυα, ποτέ να μην χαρώ.

Δεν είμαι βράχος, ούτε βουνό, 
να μην δακρύζω και να μην πονώ, 
αφού μου είπανε για σένα
κατηγορίες ένα σωρό, 
αφού μου είπανε για σένα
κατηγορ

In [137]:
stixoi_df['title'] = None

In [148]:
stixoi_df

,url,title
id,,
1,https://stixoi.info/stixoi.php?info=Lyrics&act...,1-0
2,https://stixoi.info/stixoi.php?info=Lyrics&act...,A BORD DE L΄ ASPASIA
3,https://stixoi.info/stixoi.php?info=Lyrics&act...,
4,https://stixoi.info/stixoi.php?info=Lyrics&act...,AGRIGENTO
5,https://stixoi.info/stixoi.php?info=Lyrics&act...,
...,...,...
124570,https://stixoi.info/stixoi.php?info=Lyrics&act...,None
124571,https://stixoi.info/stixoi.php?info=Lyrics&act...,None
124572,https://stixoi.info/stixoi.php?info=Lyrics&act...,None


In [168]:
#TESTS
stixoi_df.loc[113, 'title']
title_parse(stixoi_df.loc[961, 'url'])

'ΤΑ ΓΑΛΑΖΙΑ ΣΟΥ ΓΡΑΜΜΑΤΑ'

In [ ]:
#populate df with titles - Τhis does the job and takes several hours to complete since it parses around 130,000 webpages to get the titles

for index in range(min_id, max_id+1):
    url = stixoi_df.loc[index, 'url']
    try: 
        title = title_parse(url)
    except:
        title = None
    finally:
        stixoi_df.loc[index, 'title'] = title
        #print(f'index {index} : OK')

In [172]:
stixoi_df.to_csv('stixoi_info_song_titles_and_urls.csv')

In [175]:
stixoi_df.to_excel('stixoi_info_song_titles_and_urls.xlsx')

In [195]:
pd.set_option('display.max_colwidth', None) #do not truncata url values
stixoi_df.head()

,url,title
id,,
1,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=1,1-0
2,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=2,A BORD DE L΄ ASPASIA
3,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=3,
4,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=4,AGRIGENTO
5,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=5,


In [197]:
stixoi_df['url'].head()

id
1    https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=1
2    https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=2
3    https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=3
4    https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=4
5    https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=5
Name: url, dtype: object

In [198]:
stixoi_df.shape[0]

124574

We successfully extracted all urls and song titles from stixoi.info - 124,574 titles 
And saved to an excel file

In [199]:
track_title = "ΔΑΚΡΥΣΑΝ ΤΑ ΣΕΝΤΟΝΙΑ ΜΟΥ"

stixoi_df[stixoi_df['title'].str.contains(track_title, na=False)]

,url,title
id,,
99271,https://stixoi.info/stixoi.php?info=Lyrics&act=details&song_id=99271,ΔΑΚΡΥΣΑΝ ΤΑ ΣΕΝΤΟΝΙΑ ΜΟΥ


In [210]:
df.head()

,CD,track,CD path,track path,CD songs list jpg,title,singer,mp3,video title,video path,video created
0,CD01,01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR01 ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ.mp4,None
1,CD01,02,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR02 ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ.mp4,None
2,CD01,03,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/03 - ---------S S-T--S--.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR03 ΕΠΑΡΧΙΑΚΟΣ ΣΤΘΜΟΣ - Π.ΠΑΝΔΗΣ.mp4,None
3,CD01,04,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/04 - ---- S- --------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΟΤΑΝ ΣΕ ΒΡΗΚΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΟΤΑΝ ΣΕ ΒΡΗΚΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR04 ΟΤΑΝ ΣΕ ΒΡΗΚΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ.mp4,None
4,CD01,05,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/05 - ------S -- --- ------ -----.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΕΠΑΙΞΕΣ ΜΕ ΤΗΝ ΚΑΡΔΙΑ ΜΟΥ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΕΠΑΙΞΕΣ ΜΕ ΤΗΝ ΚΑΡΔΙΑ ΜΟΥ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR05 ΕΠΑΙΞΕΣ ΜΕ ΤΗΝ ΚΑΡΔΙΑ ΜΟΥ - Π.ΠΑΝΔΗΣ.mp4,None


In [251]:
def parse_lyrics_and_save(df):
    """
    Takes in df with songs info
    Returns: populated df with description of songs including lyrics if available - videos are then ready to share in YouTube
    """
    count = 0 #will count how many songs from the 1000 have been found with lyrics from stixoi.info
    import pprint
    for i in range(100):
        current_track = df.loc[i]
        current_title = df.loc[i, 'title'].text[0] #get the title of the current song out of 1000 of Mikis Theodorakis
        stixoi_subdf = stixoi_df[stixoi_df['title'].str.contains(current_title, na=False)] #find the songs in stixoi.info that contain this title as their title
        if not stixoi_subdf.empty: #if it is not empyt, i.e. a result was returned
            count += 1
            current_lyrics_selected = stixoi_subdf.iloc[0]['url']#get the first result found if multiple matches are located
            #pprint.pprint(current_title)
            #print(f'selected {current_lyrics_selected}')
            #print('----------\n')
    
    return(count)

    """         
    with open('lyrics_sample.txt', 'w') as f:
    for item in a_string:
        print(item)
        f.write(item.replace("\n",""))
    """

parse_lyrics_and_save(df)

48

In [283]:
counter = 0
for i in range(len(df)):
    if os.path.isfile(df.loc[i, 'video path']):
        counter += 1
        
print(counter)

817


out of the 1000 songs only 48 were found with lyrics matching their title, hence there is no point to further investigate getting the lyrics automatically from stixoi.info

# Upload to YouTube

In [286]:
for i in range(len(df)):
    if os.path.isfile(df.loc[i, 'video path']):
        df.loc[i, 'video created'] = True
        
df.head(2)

,CD,track,CD path,track path,CD songs list jpg,title,singer,mp3,video title,video path,video created,video description
0,CD01,01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ,Π.ΠΑΝΔΗΣ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR01 ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ.mp4,True,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 01\n Title: ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: Π.ΠΑΝΔΗΣ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.
1,CD01,02,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"[C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg, C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,"[TPE2, TIT2, TALB, TPE1, TRCK, TSSE, TENC, TSOC, COMM::eng, APIC:, COMM:ID3v1 Comment:eng]",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR02 ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ.mp4,True,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 02\n Title: ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.


In [295]:
df.to_excel("df_saved.xlsx")

In [296]:
test_df = pd.read_excel("df_saved.xlsx")
test_df.head(2)

Unnamed: 0    CD  track  \
0           0  CD01      1   
1           1  CD01      2   

                                                                                         CD path  \
0  C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01   
1  C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01   

                                                                                                                                   track path  \
0      C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3   
1  C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3   

                                                                                                                                                                                                            CD songs list jpg  \
0  ['C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/1000  Cover.jpg', 'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/CD1.jpg']   
1  ['C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/1000  Cover.jpg', 'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS  52 CD/Mikis Theodorakis CD01/CD1.jpg']   

                    title          singer  \
0  ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ        Π.ΠΑΝΔΗΣ   
1    ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ  ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [302]:
test_df.loc[0, 'video description']

'1000 τραγούδια του Μίκη Θεοδωράκη\n                        CD01\n                        Track 01\n                        Title: ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ\n                        Music: Μίκης Θεοδωράκης\n                        Singer: Π.ΠΑΝΔΗΣ\n\n                        Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων                        της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n                        ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.'

# Read the excel file with the dataframe of the video_songs and extract its values

In [308]:
df = pd.read_excel('results_of_script.xlsx')
df.drop(['Unnamed: 0', 'mp3'], axis=1, inplace=True) 
df.head(2)

,CD,track,CD path,track path,CD songs list jpg,title,singer,video title,video description,video path,video created
0,CD01,1,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/01 - -- ----- ---- - --------.-----S.mp3,"['C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg', 'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg']",ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ,Π.ΠΑΝΔΗΣ,ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 01\n Title: ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: Π.ΠΑΝΔΗΣ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR01 ΜΕ ΒΡΗΚΕ ΠΑΛΙ Η ΕΡΗΜΙΑ - Π.ΠΑΝΔΗΣ.mp4,True
1,CD01,2,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01,C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/02 - S- ---S-O S-- ---------.--G----G---.mp3,"['C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/1000 Cover.jpg', 'C:/Users/User/Downloads/Mp3toMp4/Audio_tracks/THEODORAKIS MIKIS 52 CD/Mikis Theodorakis CD01/CD1.jpg']",ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ,ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ,1000 τραγούδια του Μίκη Θεοδωράκη\n CD01\n Track 02\n Title: ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ\n Music: Μίκης Θεοδωράκης\n Singer: ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ\n\n Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.,C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR02 ΣΕ ΒΡΙΣΚΩ ΣΤΑ ΒΙΒΛΙΑ - ΑΛ.ΚΑΓΙΑΛΟΓΛΟΥ.mp4,True


In [315]:
CD_selected = 'CD01'

sub_df = df[df['CD'] == 'CD01']
print(f'Ready to upload {sub_df.shape[0]} videos for {CD_selected}')
video_keywords = f"{sub_df.loc[i, 'CD']}, Mikis Theodorakis, Μίκης Θεοδωράκης, 1000 songs, 1000 τραγούδια"

for i in range(len(sub_df)): #for each video track within the selected CD
    video_path = sub_df.loc[i, 'video path']
    video_title = f"{sub_df.loc[i, 'video title']} - {sub_df.loc[i, 'CD']} TR{sub_df.loc[i, 'track']}"
    video_description = sub_df.loc[i, 'video description']

video_path, video_title, video_description, video_keywords

Ready to upload 18 videos for CD01


('C:/Users/User/Downloads/Mp3toMp4/Video_tracks/CD01TR18 ΧΑΪ - ΧΑΪ-Χ.ΑΛΕΞΙΟΥ.mp4',
 'ΧΑΪ - ΧΑΪ-Χ.ΑΛΕΞΙΟΥ - CD01 TR18',
 '1000 τραγούδια του Μίκη Θεοδωράκη\n                        CD01\n                        Track 18\n                        Title: ΧΑΪ\n                        Music: Μίκης Θεοδωράκης\n                        Singer: ΧΑΪ-Χ.ΑΛΕΞΙΟΥ\n\n                        Πραγματοποιείται μια προσπάθεια να ανέβουν στο youtube και τα περίπου 1000 τραγούδια του Μίκη Θεοδωράκη που συγκεντρώθηκαν από μια ομάδα φίλων                        της μουσικής του Μίκη στο http://mikis1000songs.blogspot.gr/ ώστε να φτάσουν τα τραγούδια του σε κάθε γωνιά της ελληνικής γης και σε κάθε πτυχή της ελληνικής ψυχής.\n\n                        ΑΠΟΠΟΙΗΣΗ: Το οπτικό και ηχητικό υλικό του βίντεο δε μου ανήκει. Το παρόν δημιουργήθηκε μόνο για λόγους ψυχαγωγίας και δεν αποσκοπεί σε παραβίαση πνευματικών δικαιωμάτων.',
 'CD01, Mikis Theodorakis, Μίκης Θεοδωράκης, 1000 songs, 1000 τραγούδια')